In [329]:
import pandas as pd
from sklearn.tree import DecisionTreeRegressor , export_graphviz
import numpy as np

In [330]:
data_file = '../train.csv'

In [331]:
data = pd.read_csv(train_file)

In [332]:
data.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


In [333]:
data.describe()

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
count,10886.000000,10886.000000,10886.000000,10886.000000,10886.00000,10886.000000,10886.000000,10886.000000,10886.000000,10886.000000,10886.000000
mean,2.506614,0.028569,0.680875,1.418427,20.23086,23.655084,61.886460,12.799395,36.021955,155.552177,191.574132
std,1.116174,0.166599,0.466159,0.633839,7.79159,8.474601,19.245033,8.164537,49.960477,151.039033,181.144454
min,1.000000,0.000000,0.000000,1.000000,0.82000,0.760000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,2.000000,0.000000,0.000000,1.000000,13.94000,16.665000,47.000000,7.001500,4.000000,36.000000,42.000000
50%,3.000000,0.000000,1.000000,1.000000,20.50000,24.240000,62.000000,12.998000,17.000000,118.000000,145.000000
75%,4.000000,0.000000,1.000000,2.000000,26.24000,31.060000,77.000000,16.997900,49.000000,222.000000,284.000000
max,4.000000,1.000000,1.000000,4.000000,41.00000,45.455000,100.000000,56.996900,367.000000,886.000000,977.000000


In [334]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 12 columns):
datetime      10886 non-null object
season        10886 non-null int64
holiday       10886 non-null int64
workingday    10886 non-null int64
weather       10886 non-null int64
temp          10886 non-null float64
atemp         10886 non-null float64
humidity      10886 non-null int64
windspeed     10886 non-null float64
casual        10886 non-null int64
registered    10886 non-null int64
count         10886 non-null int64
dtypes: float64(3), int64(8), object(1)
memory usage: 1020.6+ KB


Несложно понять что casual + registered = count, так что эти два столбца я предлагаю убрать(то есть они высоко коррелируют).Также преобразуем datetime в числовые столбцы mounth, day, hour.

In [335]:
data.datetime = data.datetime.apply(pd.to_datetime)
data['month'] = data.datetime.apply(lambda x: x.month)
data['hour'] = data.datetime.apply(lambda x: x.hour)
data['day'] = data.datetime.apply(lambda x: x.day)
data.drop(['datetime'], axis = 1, inplace = True)
data.drop(['casual', 'registered'], axis = 1, inplace = True)
data.head()

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,count,month,hour,day
0,1,0,0,1,9.84,14.395,81,0.0,16,1,0,1
1,1,0,0,1,9.02,13.635,80,0.0,40,1,1,1
2,1,0,0,1,9.02,13.635,80,0.0,32,1,2,1
3,1,0,0,1,9.84,14.395,75,0.0,13,1,3,1
4,1,0,0,1,9.84,14.395,75,0.0,1,1,4,1


Проверим что все признаки численные.

In [336]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 12 columns):
season        10886 non-null int64
holiday       10886 non-null int64
workingday    10886 non-null int64
weather       10886 non-null int64
temp          10886 non-null float64
atemp         10886 non-null float64
humidity      10886 non-null int64
windspeed     10886 non-null float64
count         10886 non-null int64
month         10886 non-null int64
hour          10886 non-null int64
day           10886 non-null int64
dtypes: float64(3), int64(9)
memory usage: 1020.6 KB


In [337]:
x = data.drop(['count'], axis = 1)
y = data['count']
x.shape , y.shape

((10886, 11), (10886,))

Построим модель и посмотрим на результат.

In [338]:
from sklearn.model_selection import train_test_split, cross_val_score
x_train , x_test , y_train , y_test = train_test_split(x, y, test_size = 0.3, random_state = 11)
x_train.shape , x_test.shape , y_train.shape , y_test.shape

((7620, 11), (3266, 11), (7620,), (3266,))

In [339]:
model_tree = DecisionTreeRegressor(max_depth = 6, random_state = 11)
model_tree.fit(x_train, y_train)

DecisionTreeRegressor(criterion='mse', max_depth=6, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=11, splitter='best')

In [340]:
model_tree.score(x_test, y_test)

0.687769791030726

Настроим гиперпараметр.

In [341]:
from sklearn.model_selection import GridSearchCV
tree_param = {'max_depth':range(1,10), 'max_features': [1, 0.7, 0.5]}

In [342]:
tree_grid = GridSearchCV(model_tree,tree_param,cv = 5)

In [343]:
tree_grid.fit(x_train,y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=DecisionTreeRegressor(criterion='mse', max_depth=6,
                                             max_features=None,
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             presort=False, random_state=11,
                                             splitter='best'),
             iid='warn', n_jobs=None,
             param_grid={'max_depth': range(1, 10),
                         'max_features': [1, 0.7, 0.5]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbo

In [344]:
tree_grid.best_score_ , tree_grid.best_params_

(0.7456979984664884, {'max_depth': 9, 'max_features': 0.7})

In [345]:
tree_grid.fit(x_train,y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=DecisionTreeRegressor(criterion='mse', max_depth=6,
                                             max_features=None,
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             presort=False, random_state=11,
                                             splitter='best'),
             iid='warn', n_jobs=None,
             param_grid={'max_depth': range(1, 10),
                         'max_features': [1, 0.7, 0.5]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbo

In [346]:
tree_grid.score(x_test, y_test)

0.762568825812936

In [348]:
export_graphviz(tree_grid.best_estimator_ , out_file = 'Treebest.dot' , feature_names = x.columns , filled = True)

In [350]:
export_graphviz(model_tree, out_file = 'Tree.dot', feature_names = x.columns , filled = True)